# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [1]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [2]:
sms_spam_df = spark.read.csv('sms_spam.csv', header=True, inferSchema=True)

# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`. Besides, assign the count of spam to `spam_count` and the count of ham to `ham_count`. 

In [3]:
# create sms_spam2_df below
# YOUR CODE HERE
#raise NotImplementedError()
sms_spam2_df = sms_spam_df.select(fn.when(fn.col('type')=='spam',1).otherwise(0).alias('type'),fn.col('text'))
spam_count = sms_spam2_df.select(fn.sum(fn.when(fn.col('type') == '1.0',1))).collect()
ham_count = sms_spam2_df.select(fn.sum(fn.when(fn.col('type') == '0',1))).collect()                       

In [4]:
# check here
sms_spam2_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [5]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747])
np.testing.assert_array_equal(spam_count, 747)
np.testing.assert_array_equal(ham_count, 4827)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [6]:
# create a Pipeline transformer and name it tfidf_pipeline
# YOUR CODE HERE
from pyspark.ml.feature import Tokenizer,CountVectorizer,IDF
tokenizer = Tokenizer(inputCol= "text",outputCol="words")
countvectorizer = CountVectorizer(inputCol="words",outputCol="tf")
idf = IDF(inputCol = "tf",outputCol="tfidf")
tfidf_pipeline = Pipeline(stages=[tokenizer,countvectorizer,idf]).fit(sms_spam2_df)
#raise NotImplementedError()

In [7]:
# Check pipeline result
tfidf_pipeline.transform(sms_spam2_df).show()

+----+--------------------+--------------------+--------------------+--------------------+
|type|                text|               words|                  tf|               tfidf|
+----+--------------------+--------------------+--------------------+--------------------+
|   0|Go until jurong p...|[go, until, juron...|(13525,[8,42,51,6...|(13525,[8,42,51,6...|
|   0|Ok lar... Joking ...|[ok, lar..., joki...|(13525,[5,74,404,...|(13525,[5,74,404,...|
|   1|Free entry in 2 a...|[free, entry, in,...|(13525,[0,3,8,20,...|(13525,[0,3,8,20,...|
|   0|U dun say so earl...|[u, dun, say, so,...|(13525,[5,22,60,1...|(13525,[5,22,60,1...|
|   0|Nah I don't think...|[nah, i, don't, t...|(13525,[0,1,66,86...|(13525,[0,1,66,86...|
|   1|FreeMsg Hey there...|[freemsg, hey, th...|(13525,[0,2,6,10,...|(13525,[0,2,6,10,...|
|   0|Even my brother i...|[even, my, brothe...|(13525,[0,7,9,13,...|(13525,[0,7,9,13,...|
|   0|As per your reque...|[as, per, your, r...|(13525,[0,10,11,4...|(13525,[0,10,11,4...|

In [8]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [9]:
# create sms_spam3_df below
# YOUR CODE HERE
from pyspark.sql.functions import regexp_extract,col,length
sms_spam3_df = sms_spam2_df.select('type','text',regexp_extract(col('text'),'([A-Z]{3,})',1).alias('has_uppercase')) # generating  a new column has_uppercase for uppercase starting text
sms_spam3_df = sms_spam3_df.select('type','text',fn.when(fn.length(fn.col('has_uppercase')) >=3,1).otherwise(0).alias('has_uppercase')) # checking for length >3 and giving it a value 1.
#raise NotImplementedError()

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [10]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]

In [11]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [12]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [13]:
[training_df.count(), validation_df.count(), testing_df.count()]

[3311, 1709, 554]

**(15 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$) as following:
$$\lambda = [0, 0.02, 0.1] $$
$$\alpha = [0.2, 0.4] $$

Try different combinations of $\lambda$ and $\alpha$ (6 combinations) to find out best parameters by using the area under the curve as the estimator. Fit those pipelines to the appropriate data split and assign the pipeline with the best model to a variable `best_model`. Also, assign $\lambda$ and $\alpha$ of the best model to `best_model_lambda` and `best_model_alpha`.

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [14]:
# Testing all the combinations of alpha and lambda and assign the best one to `best_model` based on AUC.
# YOUR CODE HERE
# Generating models based on all 6 variations of alpha and lambda
m1_pipeline = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression(regParam=0, elasticNetParam=0.2, labelCol = 'type', featuresCol = 'tfidf')]).fit(training_df)
m2_pipeline = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression(regParam=0, elasticNetParam=0.4, labelCol = 'type', featuresCol = 'tfidf')]).fit(training_df)
m3_pipeline = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression(regParam=0.02, elasticNetParam=0.2, labelCol = 'type', featuresCol = 'tfidf')]).fit(training_df)
m4_pipeline = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression(regParam=0.02, elasticNetParam=0.4, labelCol = 'type', featuresCol = 'tfidf')]).fit(training_df)
m5_pipeline = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression(regParam=0.1, elasticNetParam=0.2, labelCol = 'type', featuresCol = 'tfidf')]).fit(training_df)
m6_pipeline = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression(regParam=0.1, elasticNetParam=0.4, labelCol = 'type', featuresCol = 'tfidf')]).fit(training_df)

#raise NotImplementedError()

In [15]:
#valuating AUC for all 6 models and picking out the highest valued area.
from pyspark.ml.evaluation import BinaryClassificationEvaluator
evaluator = evaluation.BinaryClassificationEvaluator(labelCol = "type") #initializing a binary classification evaluator
evaluator_m1 = evaluator.evaluate(m1_pipeline.transform(validation_df))
evaluator_m2 = evaluator.evaluate(m2_pipeline.transform(validation_df))
evaluator_m3 = evaluator.evaluate(m3_pipeline.transform(validation_df))
evaluator_m4 = evaluator.evaluate(m4_pipeline.transform(validation_df))
evaluator_m5 = evaluator.evaluate(m5_pipeline.transform(validation_df))
evaluator_m6 = evaluator.evaluate(m6_pipeline.transform(validation_df))

print("Model1, lambda= 0 ,alpha = 0.2, AUC = ",evaluator_m1)
print("Model2, lambda= 0 ,alpha = 0.4, AUC = ",evaluator_m2)
print("Model3, lambda= 0.02 ,alpha = 0.2, AUC = ",evaluator_m3)
print("Model4, lambda= 0.02 ,alpha = 0.4, AUC = ",evaluator_m4)
print("Model5, lambda= 0.1 ,alpha = 0.2, AUC = ",evaluator_m5)
print("Model6, lambda= 0.1 ,alpha = 0.4, AUC = ",evaluator_m6)


Model1, lambda= 0 ,alpha = 0.2, AUC =  0.9606171229900157
Model2, lambda= 0 ,alpha = 0.4, AUC =  0.9606171229900157
Model3, lambda= 0.02 ,alpha = 0.2, AUC =  0.9900130378096487
Model4, lambda= 0.02 ,alpha = 0.4, AUC =  0.9847559032304789
Model5, lambda= 0.1 ,alpha = 0.2, AUC =  0.9811574677676363
Model6, lambda= 0.1 ,alpha = 0.4, AUC =  0.9543473851948426


In [16]:
best_model = m3_pipeline
best_model_lambda = 0.02
best_model_alpha = 0.2

In [17]:
# (15 pts)
np.testing.assert_equal(type(best_model), pipeline.PipelineModel)

np.testing.assert_array_equal([type(s) for s in best_model.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

np.testing.assert_equal(best_model_lambda, 0.02)
np.testing.assert_equal(best_model_alpha, 0.2)

# Question 2.5: Generalization of best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [18]:
# assign to AUC_best the AUC of the best model selected before
# YOUR CODE HERE
AUC_best = evaluator.evaluate(best_model.transform(testing_df))
#raise NotImplementedError()

In [19]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 0.9883924843423813, significant=2)

Using the same split and the best model, compute and assign `precision`, `recall` and `f1_score`. You should first count the numbers in the confusion matrix, and then compute these metrics based on the formula.

In [20]:
tn = best_model.transform(testing_df).filter('prediction = 0 AND type = 0').count()
tp = best_model.transform(testing_df).filter('prediction = 1 AND type = 1').count()
fn = best_model.transform(testing_df).filter('prediction = 0 AND type = 1').count()
fp = best_model.transform(testing_df).filter('prediction = 1 AND type = 0').count()
precision = tp/(tp + fp)
recall = tp/(tp + fn)
f1_score =  (2 * precision * recall) / (precision + recall)

In [21]:
# (5 pts)
np.testing.assert_array_almost_equal([precision, recall, f1_score],
    [1.0, 0.7976190476190477, 0.8874172185430463], decimal=2)

# Question 2.6: Inference

Use the best pipeline fitted above (`best_model`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [22]:
# create positive_words and negative_words pandas dataframe below
# YOUR CODE HERE
word = best_model.stages[0].stages[-2].vocabulary #getting wors from the vocabulary
weight = best_model.stages[-1].coefficients.toArray() # getting weights which are coefficients for the words
data = {'word':word,'weight':weight}
df4 = pd.DataFrame(data).sort_values(by=['weight'])
negative_words = df4.head(20) #getting first 20 negative weighted words
positive_words = df4.sort_values(by=['weight'],ascending = False).head(20) #getting first 20 positive weighted words
#raise NotImplementedError()

In [23]:
# examine positive vocabulary
positive_words.head()

,word,weight
7263,sexy?,0.642738
3555,widelive.com/index.,0.588182
15,call,0.537161
12237,08714712388,0.504090
81,txt,0.495005


In [24]:
# examine solutions
negative_words.head()

,word,weight
1,i,-0.183463
3332,lose.,-0.074937
3371,fightng,-0.074937
3221,dificult,-0.074937
13,me,-0.065904


The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>7263</th>
      <td>sexy?</td>
      <td>0.642738</td>
    </tr>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.588182</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.537161</td>
    </tr>
    <tr>
      <th>12237		</th>
      <td>08714712388</td>
      <td>0.504090</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.495005</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.183463</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.074937</td>
    </tr>
    <tr>
      <th>13</th>
      <td>me</td>
      <td>-0.065904</td>
    </tr>
  </tbody>
</table>

In [25]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.675741267346245, significant=1)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.7292131526997235, significant=1)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

# Question 2.7
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [26]:
# your code and comments below
# YOUR CODE HERE
!pip install git+https://github.com/daniel-acuna/pyspark_pipes.git
from pyspark.ml.feature import Tokenizer,CountVectorizer,IDF,VectorAssembler
from pyspark_pipes import pipe
tokenizer_final = Tokenizer(inputCol= "text",outputCol="words") #generating a tokenizer
countvectorizer_final = CountVectorizer(inputCol="words",outputCol="tf") # generating vectorizer
idf_final = IDF(inputCol = "tf",outputCol="tfidf") #getting tfidf column generated
assembler_final = VectorAssembler(inputCols=["has_uppercase","tfidf"],outputCol="features")
tfidf_pipeline_final = Pipeline(stages=[tokenizer_final,countvectorizer_final,idf_final,assembler_final]).fit(sms_spam3_df)
tfidf_pipeline_transformed = tfidf_pipeline_final.transform(sms_spam3_df) #getting predictions
model=pipe(feature.VectorAssembler(inputCols=['has_uppercase','tfidf']),
           feature.MaxAbsScaler(),classification.LogisticRegression(regParam=0.2, elasticNetParam=0.1, labelCol = 'type'))
model_fitted = model.fit(tfidf_pipeline_transformed) #final model
coeff = model_fitted.stages[-1].coefficients
upper_coeff = coeff.toArray()[0] #extracting the first coefficient which is for has_uppercase
max_coeff = coeff.toArray().max() #getting the max outof all coefficients
ratio = upper_coeff/max_coeff

print("has_uppercase is positively related to an SMS being SPAM, the coefficient value is : ",upper_coeff)
print("The ratio of has_uppercase coefficient and maximum coefficient is : ",ratio)
#raise NotImplementedError()

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
  Cloning https://github.com/daniel-acuna/pyspark_pipes.git to /tmp/pip-req-build-e5j2b84r
  Running command git clone -q https://github.com/daniel-acuna/pyspark_pipes.git /tmp/pip-req-build-e5j2b84r
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
has_uppercase is positively related to an SMS being SPAM, the coefficient value is :  0.9289178747599827
The ratio of has_uppercase coefficient and maximum coefficient is :  0.4617005798337619
